# AutoAI for Text API - Sample notebook

This notebooks performs the steps required to use AutoAI for Text on API Hub, which are:
- Set up the credentials
- Make a health check 
- Inspect the datasets
- Run and experiment
- Retrieve and inspect the results

### Essential installs and imports

In [ ]:
!pip install pandas
!pip install requests

In [2]:
import requests
import pandas as pd
from time import sleep
import json

### Prepare the _headers_ and base urls for the API request

In order to make requests to AutoAI for Text you need to put authentication information in the request header. The authentication credentials include Client ID and Client secret. You can get these by clicking Get trial subscription button in the top-right corner of the [AutoAI for Text API page](https://developer.ibm.com/apis/catalog/autoai4text--autoai-for-text/Introduction) and registering a free trial using your IBMid. Once your account is registered, navigate to APIs in your account's [dashboard](https://developer.ibm.com/profile/myapis). Find the subscription to AutoAI for Text service and click on it. You can expand the dropdown and see those two credentials in the Key management section.


In [3]:
headers = {
    "X-IBM-Client-Id": "", 
    "X-IBM-Client-Secret": "",
    "Accept": "application/json"
}

base_url = "https://api.ibm.com"
autoai4text_api_url = f"{base_url}/autoai4text/run"

### Preliminary health check to make sure your credentials are OK and you can access the service!

You will get a "Welcome to the AutoAI for Text service!" message. If you receive an "Authorize" message, please check your credentials.

In [4]:
health_check_url = f"{autoai4text_api_url}/health_check"
print(health_check_url)
response = requests.get(health_check_url, headers=headers)
print(response.text)

https://api.ibm.com/autoai4text/run/health_check
"Welcome to the AutoAI for Text service!"



### Inspect the data

In [5]:
trec_train = pd.read_csv("trec_train.csv")
trec_holdout = pd.read_csv("trec_holdout.csv")
print(f"Train dataset shape: {trec_train.shape}")
print(f"Holdout dataset shape: {trec_holdout.shape}")


Train dataset shape: (5452, 2)
Holdout dataset shape: (500, 2)


In [6]:
print("Train dataset's first rows")
trec_train.head()

Train dataset's first rows


,Sentence,Label
0,How did serfdom develop in and then leave Russ...,1
1,What films featured the character Popeye Doyle ?,2
2,How can I find a list of celebrities ' real na...,1
3,What fowl grabs the spotlight after the Chines...,2
4,What is the full form of .com ?,0


In [7]:
print("Holdout dataset's first rows")
trec_holdout.head()

Holdout dataset's first rows


,Sentence,Label
0,How far is it from Denver to Aspen ?,5
1,"What county is Modesto , California in ?",4
2,Who was Galileo ?,3
3,What is an atom ?,1
4,When did Hawaii become a state ?,5


### Preparing to run an experiment: set the parameters

The only required parameter is the train_file, but here we are setting all of them.

In [9]:
params = {
        "scorer_for_ranking": "accuracy",
        "time_to_train": 600,
        "models_to_include": 'cnn,bert,lstm',
        "max_evals_count": 30,
        "ranked_pipelines_count": 4,
        "text_column_index": 0,
        "label_column_index": 1,
        "label_type": 'int',
        "file_delimiter": ',',
        "validation_ratio": .1,
}

files = {
    "train_file":('trec_train.csv', open('trec_train.csv')),
    "holdout_file":('trec_holdout.csv', open('trec_holdout.csv'))
}

### Ready to run the experiment

In [ ]:
run_experiment_url = f"{autoai4text_api_url}/experiment"
print(run_experiment_url)
response = requests.post(
    run_experiment_url,
    headers=headers,
    files=files,
    params=params
)
exp_id = response.json()['exp_id']
f'The experiment ID is : {exp_id}'

### Make status checks against the above returned _exp\_id_

The below code snippet will make status check calls until the experiment results are available, i.e. the status code is 200 

In [11]:
experiment_status_url = f"{autoai4text_api_url}/experiment/{exp_id}"
print(experiment_status_url)

def get_results():
    return requests.get(experiment_status_url, headers=headers)

response = get_results()
status_code = response.status_code
sleep(600)
while status_code != 200:
    sleep(30)
    response = get_results()
    status_code = response.status_code

if status_code != 200:
    print('Unexpected error')
    print(response.text)
elif status_code == 200:
    print(f"The experiment has finished and {response.json()['message']}") 

https://api.ibm.com/autoai4text/run/experiment/ddba9c1a3d224263b453da866b654026
The experiment has finished and optimizer has learnt 4 pipelines.


### Inspect the results

#### _See the pipeline nodes_

In [14]:
results = response.json()['report']
print(f'results is a list containing {len(results)} pipeline reports')
print(f'The nodes of the best pipeline are:')
print(json.dumps(results[0]['pipeline_nodes'], indent=4))

results is a list containing 4 pipeline reports
The nodes of the best pipeline are:
{
    "TransformerStylePreprocessor": {
        "op": "TransformerStylePreprocessor",
        "hyperparameters": {
            "model_name": "BERT",
            "max_length": 200
        }
    },
    "TransformerStyleEmbedding": {
        "op": "TransformerStyleEmbedding"
    },
    "MLPClassifier": {
        "op": "MLPClassifier"
    }
}


#### _See the metric scores for validation and holdout:_

In [16]:
print(f'The validation metric scores of the best pipeline are:')
print(json.dumps(results[0]['validation_scores_report'], indent=4))
print(f'The holdout metric scores of the best pipeline are: ')
print(json.dumps(results[0]['holdout_scores_report'], indent=4))

The validation metric scores of the best pipeline are:
{
    "recall_weighted": 0.945054945054945,
    "recall_macro": 0.9016174486783936,
    "recall_micro": 0.945054945054945,
    "precision_weighted": 0.9450317425622976,
    "precision_macro": 0.9029358216301069,
    "precision_micro": 0.945054945054945,
    "f1_weighted": 0.9448581324742835,
    "f1_macro": 0.9021297306344517,
    "f1_micro": 0.945054945054945,
    "accuracy": 0.945054945054945,
    "runtime_per_row_predict": 3.1365985835428205
}
The holdout metric scores of the best pipeline are: 
{
    "recall_weighted": 0.97,
    "recall_macro": 0.9388377011308453,
    "recall_micro": 0.97,
    "precision_weighted": 0.9707023257930742,
    "precision_macro": 0.9765440054758812,
    "precision_micro": 0.97,
    "f1_weighted": 0.9693295945543323,
    "f1_macro": 0.9547405298858841,
    "f1_micro": 0.97,
    "accuracy": 0.97,
    "runtime_per_row_predict": 2.414210319519043
}


#### _See the predictions made by the best pipeline from the holdout dataset_

In [17]:
print(f'The predictions on the holdout dataset of the best pipeline are:')
print(results[0]['holdout_predicted_values'])

The predictions on the holdout dataset of the best pipeline are:
[5.0, 4.0, 3.0, 1.0, 5.0, 5.0, 3.0, 2.0, 1.0, 1.0, 4.0, 3.0, 5.0, 3.0, 5.0, 5.0, 2.0, 3.0, 1.0, 5.0, 3.0, 1.0, 4.0, 1.0, 1.0, 3.0, 1.0, 4.0, 4.0, 4.0, 5.0, 4.0, 1.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 5.0, 1.0, 5.0, 3.0, 1.0, 3.0, 3.0, 1.0, 1.0, 1.0, 5.0, 4.0, 4.0, 5.0, 4.0, 3.0, 4.0, 2.0, 4.0, 3.0, 2.0, 1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 3.0, 4.0, 1.0, 2.0, 5.0, 5.0, 3.0, 1.0, 5.0, 3.0, 5.0, 5.0, 1.0, 1.0, 3.0, 1.0, 4.0, 2.0, 1.0, 5.0, 5.0, 3.0, 4.0, 5.0, 1.0, 1.0, 5.0, 1.0, 3.0, 1.0, 3.0, 4.0, 1.0, 5.0, 2.0, 5.0, 4.0, 2.0, 1.0, 5.0, 3.0, 2.0, 3.0, 5.0, 2.0, 5.0, 4.0, 5.0, 5.0, 1.0, 3.0, 5.0, 3.0, 5.0, 5.0, 1.0, 5.0, 5.0, 3.0, 1.0, 2.0, 1.0, 4.0, 4.0, 3.0, 3.0, 1.0, 1.0, 2.0, 4.0, 2.0, 1.0, 2.0, 3.0, 2.0, 1.0, 1.0, 2.0, 3.0, 1.0, 5.0, 3.0, 4.0, 4.0, 1.0, 4.0, 4.0, 1.0, 1.0, 5.0, 4.0, 2.0, 4.0, 5.0, 1.0, 4.0, 3.0, 5.0, 5.0, 5.0, 1.0, 4.0, 4.0, 4.0, 5.0, 2.0, 5.0, 4.0, 1.0, 4.0, 1.0, 2.0, 3.0, 3.0,